In [5]:
import boto
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
from fnmatch import fnmatch

import re
import json
import glob
from random import shuffle

from pytz import country_timezones

import pytz

import numpy as np

from bokeh.plotting import figure, show, output_file
from datalab.context import Context

# CODE TO IMPORT LOGS FROM S3 TO GCP USING BATCH OF 10000 LOGS

In [6]:
#connect with S3
AWS_KEY = <YOUR_AWS_KEY>
AWS_SECRET = <YOUR_AWS_SECRET>
conn = boto.connect_s3(AWS_KEY,AWS_SECRET)

In [7]:
#Get specific bucket
specific_bucket = conn.get_bucket(<YOUR_BUCKET>)

#Get files from specific prefix
#bucket_files = specific_bucket.list(prefix='2016/08/01/4155902955122327623')
bucket_files = specific_bucket.list(prefix='2016/10/20')

In [8]:
b_files = [log_file for log_file in bucket_files]

In [9]:
#Get only files without bug recordings
new_files = b_files[0:68517]

In [24]:
subset_files = new_files[20000:30000]

conc_contents = "["

for log_file in subset_files:
    
    if subset_files.index(log_file) !=0:
      conc_contents = conc_contents + ','  
      
    preconts = log_file.read()
    pre_cont1 = preconts.strip('\n')
    pre_cont2 = re.split(r"}\n{",pre_cont1)
    pre_cont3 = '},{'.join(pre_cont2)
    conc_contents = conc_contents + pre_cont3
conc_contents = conc_contents + "]"

In [25]:
ok_contents = json.loads(conc_contents) 
#Converting to DataFrame
df_contents = pd.DataFrame(ok_contents)

In [26]:
df = df_contents.apply(lambda x : tuple(x) if type(x) is list else x)

In [27]:
#making preprocessing to handle df methods
features = ['error_source','get','post','referrer_user','referrer_user_guid_all','referring_user','user']

features = [column for column in df.columns if column in features]

for feature in features:
  df[feature] = df[feature].apply(lambda x : tuple(x) if type(x) is list else x)
#df.user.apply(lambda x : tuple(x) if type(x) is list else x)

In [28]:
#droping irrelevant features
drop_features = ['app_release','bluetooth_enabled','bluetooth_version',
                'entity_type','event','forced_login','get',
                'guid_game','handle_url','nfc','post','referrer_user',
                'referrer_user_guid_all','referring_user','server']

drop_features = [column for column in df.columns if column in drop_features]

df = df.drop(drop_features,axis=1)

In [29]:
df = df.drop_duplicates()

In [30]:
df = df.dropna(axis=1,how='all')

In [31]:
df = df[df['_type'].str.startswith('slow:') == False]

In [32]:
#Converting timestamp into date/time 
time_conv = pd.to_datetime(df['time'],unit='s')
time_data = pd.DatetimeIndex(time_conv)

In [33]:
df['day'] = time_data.day
df['dayofweek'] = time_data.dayofweek
df['dayofyear'] = time_data.dayofyear
df['month'] = time_data.month
df['daysinmonth'] = time_data.daysinmonth
df['year'] = time_data.year
df['hour'] = time_data.hour
df['minutes'] = time_data.minute
df['seconds'] = time_data.second
df['weekofyear'] = time_data.weekofyear
df['time_data'] = time_data

In [34]:
#Every empty field will be filled with 777 code
#df = df.replace([np.NAN],[777])

In [35]:
json_file = df.to_json()

In [1]:
project = Context.default().project_id
bucket_name = <YOUR_BUCKET>
bucket_path = 'gs://' + bucket_name + <YOUR_PREFIX>
bucket_object = bucket_path + 'YOUR_LOG_FILE'

print 'Bucket: ' + bucket_path
print 'Object: ' + bucket_object

SyntaxError: invalid syntax (<ipython-input-1-8c58854e48ea>, line 2)

In [37]:
%%storage write --variable json_file --object $bucket_object

#Fields that need to be preprocessed from the dataset.
#error_source,get,post (check in more detail), referrer_user (check in more detail), referrer_user_guid_all (check in more detail),referring_user,user